In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./super_store.csv")

In [3]:
print(df.size)

176400


In [4]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales'],
      dtype='object')

In [5]:
df_copy = df.copy()
print(df_copy)


      Row ID        Order ID  Order Date   Ship Date       Ship Mode  \
0          1  CA-2017-152156  08/11/2017  11/11/2017    Second Class   
1          2  CA-2017-152156  08/11/2017  11/11/2017    Second Class   
2          3  CA-2017-138688  12/06/2017  16/06/2017    Second Class   
3          4  US-2016-108966  11/10/2016  18/10/2016  Standard Class   
4          5  US-2016-108966  11/10/2016  18/10/2016  Standard Class   
...      ...             ...         ...         ...             ...   
9795    9796  CA-2017-125920  21/05/2017  28/05/2017  Standard Class   
9796    9797  CA-2016-128608  12/01/2016  17/01/2016  Standard Class   
9797    9798  CA-2016-128608  12/01/2016  17/01/2016  Standard Class   
9798    9799  CA-2016-128608  12/01/2016  17/01/2016  Standard Class   
9799    9800  CA-2016-128608  12/01/2016  17/01/2016  Standard Class   

     Customer ID     Customer Name    Segment        Country             City  \
0       CG-12520       Claire Gute   Consumer  United 

In [50]:
print((df.dtypes).replace("object", "string"))

Row ID             int64
Order ID          string
Order Date        string
Ship Date         string
Ship Mode         string
Customer ID       string
Customer Name     string
Segment           string
Country           string
City              string
State             string
Postal Code      float64
Region            string
Product ID        string
Category          string
Sub-Category      string
Product Name      string
Sales            float64
dtype: object


## Ship Mode Dimension table creation
In this section, the following tasks have been performed:
1. Create a dimension table "dim_ship_mode" for ship mode column.
2. Map each ship mode unique value an ID.
3. Replace the name of the column "Ship Mode" in the main table to "sp_id".
4. Map each ID from "dim_ship_mode" to the main table "sp_id" and replace them.

In [6]:
#Assigning ship mode and ids
df["Ship Mode"].unique()
dim_ship_mode = pd.DataFrame({"sp_id" : (i[:2] for i in df["Ship Mode"].unique()), "ship_mode" : df["Ship Mode"].unique()})
#Assigning unique IDS
dim_ship_mode["sp_id"] = dim_ship_mode["sp_id"].str.upper()
print(dim_ship_mode)

  sp_id       ship_mode
0    SE    Second Class
1    ST  Standard Class
2    FI     First Class
3    SA        Same Day


In [7]:
df_copy.rename(columns = {"Ship Mode": "sp_id"}, inplace = True)
print(df_copy.head())

   Row ID        Order ID  Order Date   Ship Date           sp_id Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal Code Region       Product ID         Cat

In [8]:
#replace every entry with id
df_copy.replace(np.asarray(dim_ship_mode["ship_mode"]), dim_ship_mode["sp_id"], inplace = True)

In [9]:
#Exporting to csv file
dir_name = "./Dimension-tables/"
dim_ship_mode.to_csv((dir_name+ "dim_ship_mode.csv"), index = False)

## Customer Information Dimension


In [10]:
dim_customer_info = pd.DataFrame({"customer_id": df_copy["Customer ID"], "customer_name": df_copy.pop("Customer Name")})

In [11]:
cust_id = dim_customer_info["customer_id"].unique()
cust_name = dim_customer_info["customer_name"].unique()
dim_customer_info = pd.DataFrame({"customer_id": cust_id, "customer_name": cust_name})

In [12]:
print(dim_customer_info)
df_copy.rename(columns = {"Customer ID": "customer_id"}, inplace = True)

    customer_id      customer_name
0      CG-12520        Claire Gute
1      DV-13045    Darrin Van Huff
2      SO-20335     Sean O'Donnell
3      BH-11710    Brosina Hoffman
4      AA-10480       Andrew Allen
..          ...                ...
788    CJ-11875       Carl Jackson
789    RS-19870         Roy Skaria
790    SC-20845         Sung Chung
791    RE-19405    Ricardo Emerson
792    SM-20905  Susan MacKendrick

[793 rows x 2 columns]


In [13]:
df_copy

,Row ID,Order ID,Order Date,Ship Date,sp_id,customer_id,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,SE,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,SE,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,SE,DV-13045,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,ST,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,ST,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,ST,SH-19975,Corporate,United States,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,Corporate,United States,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [14]:
dim_customer_info.to_csv((dir_name + "dim_customer_info.csv"), index = False)

## Segment Dimensions table:

In [15]:
df_copy["Segment"].unique()

array(['Consumer', 'Corporate', 'Home Office'], dtype=object)

In [16]:
dim_segment = pd.DataFrame(
    {
        "seg_id":["CON", "COR", "HOF"],
        "segment":["Consumer", "Corporate", "Home Office"]
    }
)

In [17]:
dim_segment.to_csv((dir_name + "dim_segment.csv"), index = False)

In [18]:
#Replacing in the main table
df_copy.rename({"Segment":"seg_id"},axis = 1, inplace = True)
df_copy.replace(np.asarray(dim_segment["segment"]), dim_segment["seg_id"], inplace = True)

## Country 

In [19]:
df_country = pd.read_csv("./country_code.csv")

In [20]:
df_country.drop(["Alpha-2 code", "Numeric"], axis = 1, inplace = True)
df_country.rename({"Alpha-3 code": "country_id"}, axis = 1, inplace = True)

In [21]:
df_country = df_country.loc[:,['country_id','Country']]
df_country.rename(columns = {
    'Country':"country_name"
}, inplace = True)


In [22]:
df_copy.replace(np.asarray(df_country["country_name"]), df_country["country_id"], inplace = True)
df_copy.rename({
    "Country":"country_id"
},axis = 1, inplace =True)

In [23]:
df_copy

,Row ID,Order ID,Order Date,Ship Date,sp_id,customer_id,seg_id,country_id,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,SE,CG-12520,CON,US,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,SE,CG-12520,CON,US,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,SE,DV-13045,COR,US,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,ST,SO-20335,CON,US,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,ST,SO-20335,CON,US,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,ST,SH-19975,COR,US,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [24]:
df_country.to_csv((dir_name + "dim_country.csv"), index = False)

## States

In [25]:
df_states = pd.read_csv("./president.csv")
df_states

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT
1,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,504070,1182850,20210113,NaN,REPUBLICAN
2,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"MADDOX, LESTER",AMERICAN INDEPENDENT PARTY,False,9198,1182850,20210113,NaN,OTHER
3,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"BUBAR, BENJAMIN """"BEN""""",PROHIBITION,False,6669,1182850,20210113,NaN,OTHER
4,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"HALL, GUS",COMMUNIST PARTY USE,False,1954,1182850,20210113,NaN,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4282,2020,WYOMING,WY,56,83,68,US PRESIDENT,"JORGENSEN, JO",LIBERTARIAN,False,5768,278503,20210113,NaN,LIBERTARIAN
4283,2020,WYOMING,WY,56,83,68,US PRESIDENT,"PIERCE, BROCK",INDEPENDENT,False,2208,278503,20210113,NaN,OTHER
4284,2020,WYOMING,WY,56,83,68,US PRESIDENT,NaN,NaN,True,1739,278503,20210113,NaN,OTHER
4285,2020,WYOMING,WY,56,83,68,US PRESIDENT,OVERVOTES,NaN,False,279,278503,20210113,NaN,OTHER


In [26]:
state_id = df_states["state_po"].unique()
state_name_Caps = df_states["state"].unique()
state_name = []
for st_nm in state_name_Caps:
    state_name.append(st_nm.title())

df_states = pd.DataFrame({
    "state_id": state_id,
    "state_name": state_name
})


In [27]:
state_id

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [28]:
df_states.to_csv((dir_name + "dim_state.csv"), index = False)

In [29]:
df_copy.replace(state_name, state_id, inplace = True)

In [30]:
df_copy.rename(
{
    "State":"state_id"
}, axis = 1,
inplace = True
)

In [31]:
df_copy

,Row ID,Order ID,Order Date,Ship Date,sp_id,customer_id,seg_id,country_id,City,state_id,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,SE,CG-12520,CON,US,Henderson,KY,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,SE,CG-12520,CON,US,Henderson,KY,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,SE,DV-13045,COR,US,Los Angeles,CA,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,ST,SO-20335,CON,US,Fort Lauderdale,FL,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,ST,SO-20335,CON,US,Fort Lauderdale,FL,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,ST,SH-19975,COR,US,Chicago,IL,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,Toledo,OH,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,Toledo,OH,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,Toledo,OH,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [32]:
df_copy.drop(["City", "Region", "Sub-Category"], axis = 1, inplace = True)

## Category

In [33]:
category = df_copy["Category"].unique()
category_id = ["FUR", "OFS", "TEC"]

dim_category = pd.DataFrame({
    "category_id": category_id,
    "category": category
})
dim_category.to_csv((dir_name + "dim_category.csv"), index = False)
dim_category

,category_id,category
0,FUR,Furniture
1,OFS,Office Supplies
2,TEC,Technology


## Product

In [34]:
df_copy.rename({
  "Product ID": "product_id",
    "Product Name":"product_name"
},axis = 1, inplace = True)
df_copy

,Row ID,Order ID,Order Date,Ship Date,sp_id,customer_id,seg_id,country_id,state_id,Postal Code,product_id,Category,product_name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,SE,CG-12520,CON,US,KY,42420.0,FUR-BO-10001798,Furniture,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,SE,CG-12520,CON,US,KY,42420.0,FUR-CH-10000454,Furniture,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,SE,DV-13045,COR,US,CA,90036.0,OFF-LA-10000240,Office Supplies,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,ST,SO-20335,CON,US,FL,33311.0,FUR-TA-10000577,Furniture,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,ST,SO-20335,CON,US,FL,33311.0,OFF-ST-10000760,Office Supplies,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,ST,SH-19975,COR,US,IL,60610.0,OFF-BI-10003429,Office Supplies,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,OH,43615.0,OFF-AR-10001374,Office Supplies,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,OH,43615.0,TEC-PH-10004977,Technology,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,ST,CS-12490,COR,US,OH,43615.0,TEC-PH-10000912,Technology,Anker 24W Portable Micro USB Car Charger,26.3760


In [35]:
dim_product = pd.DataFrame({
    "product_id":[],
    "product_name":[],
    "category_id":[]
})
product_id = df_copy["product_id"].unique()

for prod_id in product_id:
    dim_product.loc[len(dim_product)] = [prod_id, 
                                         df_copy[df_copy["product_id"] == prod_id]["product_name"].unique()[0],
                                         df_copy[df_copy["product_id"] == prod_id]["Category"].unique()[0]]
    

In [36]:
dim_product

,product_id,product_name,category_id
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies
...,...,...,...
1856,TEC-AC-10002380,Sony 8GB Class 10 Micro SDHC R40 Memory Card,Technology
1857,TEC-PH-10002817,RCA ViSYS 25425RE1 Corded phone,Technology
1858,TEC-MA-10003589,Cisco 8961 IP Phone Charcoal,Technology
1859,OFF-AP-10003099,"Eureka Hand Vacuum, Bagless",Office Supplies


In [37]:
dim_product.replace(
    np.asarray(dim_category["category"]),dim_category["category_id"],inplace = True)

In [38]:
dim_product.to_csv((dir_name + "dim_product_detail.csv"), index = False)

In [39]:
df_copy.rename({
    "Row ID": "row_id",
    "Order ID":"order_id",
    "Order Date":"order_date",
    "Ship Date":"ship_date",
    "Postal Code": "postal code",
    "Sales": "sales"
}, axis = 1, inplace = True)

In [40]:
df_copy.to_csv((dir_name + "fact_sales.csv"), index = False)